In [1]:
import pandas as pd
import numpy as np

In [2]:
"""
case1: df中某列存在序列特征，想要将列中的内容进行合并。
"""

df = pd.DataFrame({
    'sid': ['session1', 'session1', 'session1', 'session2', 'session2'],
    'content': ['A', 'B', 'C', 'D', 'E']
})
df

,sid,content
0,session1,A
1,session1,B
2,session1,C
3,session2,D
4,session2,E


In [3]:
# 利用group + transform
df['target'] = df.groupby(['sid'])['content'].transform(lambda x: '|'.join(x))
df

,sid,content,target
0,session1,A,A|B|C
1,session1,B,A|B|C
2,session1,C,A|B|C
3,session2,D,D|E
4,session2,E,D|E


In [4]:
# 利用drop_duplicates()去重 [重要！！]
res = df[['sid', 'target']].drop_duplicates().reset_index(drop=True)
res

,sid,target
0,session1,A|B|C
1,session2,D|E


In [5]:
"""
case1.2: 如何将case1的结果复原，如果序列长度限制为2该如何处理
"""

res

,sid,target
0,session1,A|B|C
1,session2,D|E


In [27]:
# 先利用split将目标列分开
df_split = res['target'].str.split('|', expand=True)
df_split

,0,1,2
0,A,B,C
1,D,E,None


In [28]:
# level 这个参数值得注意（丢掉index）
df_split = df_split.stack().reset_index(level=1, drop=True)
df_split

0    A
0    B
0    C
1    D
1    E
dtype: object

In [30]:
df_split.name = 'tmp'

In [35]:
# join默认使用index进行拼接
res.join(df_split)

,sid,target,tmp
0,session1,A|B|C,A
0,session1,A|B|C,B
0,session1,A|B|C,C
1,session2,D|E,D
1,session2,D|E,E


In [38]:
"""
固定序列长度为2
"""

def expand(_df):
    ss = _df['target'].split('|')
    
    res = []
    
    for i in range(len(ss) - 1):
        res.append(ss[i] + '|' + ss[i + 1])
        
    return '<SEP>'.join(res)

res['tmp_split'] = res.apply(expand, axis=1)
res

,sid,target,tmp_split
0,session1,A|B|C,A|B<SEP>B|C
1,session2,D|E,D|E


In [43]:
df_2 = res['tmp_split'].str.split('<SEP>', expand=True)
df_2 = df_2.stack().reset_index(level=1, drop=True)
df_2.name = 'series'
res_1 = res.join(df_2)
res_1[['sid', 'series']].reset_index(drop=True)

,sid,series
0,session1,A|B
1,session1,B|C
2,session2,D|E


In [17]:
"""
case2: 根据df中的两列对数据进行筛选（两列存在关联）
"""

# 想筛选出高于limit中数值的数据
limit = {'1': 0.3, '2': 0.5}
df = pd.DataFrame({
    'prdLine': ['1', '2', '2', '1', '1'],
    'score': [0.5, 0.2, 0.6, 0.1, 0.9]
})
df

,prdLine,score
0,1,0.5
1,2,0.2
2,2,0.6
3,1,0.1
4,1,0.9


In [18]:
"""
思路1: 使用apply函数生成列，再根据列筛选
"""

def judge(_df):
    line = _df['prdLine']
    score = _df['score']
    
    return score >= limit[line]

df['tmp'] = df.apply(judge, axis=1)
df

,prdLine,score,tmp
0,1,0.5,True
1,2,0.2,False
2,2,0.6,True
3,1,0.1,False
4,1,0.9,True


In [19]:
res = df[df['tmp'] == True][['prdLine', 'score']].reset_index(drop=True)
res

,prdLine,score
0,1,0.5
1,2,0.6
2,1,0.9


In [52]:
"""
case3: 将两个DataFrame进行合并，并对指定的列求和(pandas中的join)
"""

df1 = pd.DataFrame({
    'line': [1, 2, 3],
    'label_1': ['A', 'A', 'D'],
    'label_2': ['B', 'C', 'E'],
    'target_1': [2, 3, 5],
    'target_2': [4, 7, 11]
})

df1

,line,label_1,label_2,target_1,target_2
0,1,A,B,2,4
1,2,A,C,3,7
2,3,D,E,5,11


In [53]:
df2 = pd.DataFrame({
    'line': [1, 2, 3],
    'label_1': ['A', 'A', 'D'],
    'label_2': ['B', 'F', 'G'],
    'target_1': [10, 4, 3],
    'target_2': [12, 7, 6]
})

df2

,line,label_1,label_2,target_1,target_2
0,1,A,B,10,12
1,2,A,F,4,7
2,3,D,G,3,6


In [54]:
# 若原pf存在重复数据，则在merge时会出大问题
res = pd.merge(df1, df2, how='outer', on=['line', 'label_1', 'label_2'])
res = res.fillna(0)
res

,line,label_1,label_2,target_1_x,target_2_x,target_1_y,target_2_y
0,1,A,B,2.0,4.0,10.0,12.0
1,2,A,C,3.0,7.0,0.0,0.0
2,3,D,E,5.0,11.0,0.0,0.0
3,2,A,F,0.0,0.0,4.0,7.0
4,3,D,G,0.0,0.0,3.0,6.0


In [55]:
res['target_1'] = res['target_1_x'] + res['target_1_y']
res['target_2'] = res['target_2_x'] + res['target_2_y']

res = res[['label_1', 'label_2', 'target_1', 'target_2']]
res

,label_1,label_2,target_1,target_2
0,A,B,12.0,16.0
1,A,C,3.0,7.0
2,D,E,5.0,11.0
3,A,F,4.0,7.0
4,D,G,3.0,6.0
